Creating a Neural Network

In [ ]:
pip install torch


  Using cached torch-2.6.0-cp39-cp39-win_amd64.whl (204.1 MB)


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
import pandas as pd

df = pd.read_csv('final_previous_merged.csv')

df = pd.get_dummies(df, columns=["prev_currency"], prefix="dummy", drop_first=False)

df.drop(columns={'prev_X.Other.Currency.to.USD', 'prev_inf_factor'}, inplace=True)
df.columns

Index(['prev_accountsPayable', 'prev_accountsReceivable',
       'prev_bankCashBalances', 'prev_bankOverdraft',
       'prev_debitOwnedWithinOneYear', 'prev_financialAssets',
       'prev_fixedAssets', 'prev_intangibleAssets', 'prev_otherCurrentAssets',
       'prev_otherCurrentLiabilities', 'prev_otherEquity',
       'prev_otherTermAssets', 'prev_otherTermLiabilities', 'prev_prepayments',
       'prev_retainedEarnings', 'prev_subscribedCapital', 'prev_termLoans',
       'prev_totalAssets', 'prev_totalCurrentAssets',
       'prev_totalCurrentLiabilities', 'prev_totalInventories',
       'prev_totalLiabilities', 'prev_totalShareholderEquity',
       'prev_totalTermAssets', 'prev_totalTermLiabilities',
       'prev_accountPayableAccrualFromCashFlow', 'prev_capitalExpenditure',
       'prev_changeInInventoriesFromCashFlow',
       'prev_netChangeInCashAndCashEquivalents', 'prev_netFundingCashFlow',
       'prev_netInvestingCashFlow',
       'prev_unbilledAccountsReceivableRevenueFromCashF

In [4]:
from sklearn.model_selection import train_test_split

# Drop the target column for X, keep 'FHR' as y
X = df.drop("FHR", axis=1).values  # shape: (num_samples, num_features)
y = df["FHR"].values               # shape: (num_samples,)

# Split into train and test sets
#   test_size=0.2 means 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
# Convert NumPy arrays to Torch tensors (float)
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)

X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.